In [36]:
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [37]:
path = ('/content/drive/MyDrive/Colab Notebooks/hote_reviews.csv')

In [5]:
import pandas as pd
df = pd.read_csv(path)

In [6]:
df.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


**Cleaning the data**

In [7]:
df['Rating'].value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

In [8]:
df_neg = df.loc[df['Rating'] < 3]
df_neg = df_neg.reset_index(drop=True)

In [9]:
df_five = df.loc[df['Rating'] == 5]
df_five = df_five.reset_index(drop=True)

In [10]:
print(len(df_five))

9054


In [11]:
df_pos = df_five.loc[:len(df_neg)]

In [12]:
df_neg.shape

(3214, 2)

In [13]:
df_five.shape

(9054, 2)

In [14]:
df_pos.shape

(3215, 2)

In [15]:
df_all = pd.concat([df_neg, df_pos],axis=0)
df_all = df_all.reset_index(drop=True)

In [16]:
df_all.sample(5)

,Review,Rating
3862,fantastic hotel highly recommended place best ...,5
1497,3 star lobby 2 star hotel stayed start end eur...,2
4934,finest hotel experience fiance decided celebra...,5
2453,tried wo n't friend stayed caribe club princes...,2
3024,"stay away, conjure great expectations reality ...",1


**Creating sentiments column**

In [17]:
df_all['Sentiment'] = np.where(df_all['Rating'] == 5, 'Positive','Negative')

In [18]:
df_all.sample(5)

,Review,Rating,Sentiment
643,not luxurious expect wife paid 300/night stay ...,2,Negative
1012,"horrible experience not stay, car broken lot s...",1,Negative
1897,terrible service beautiful beaches recently we...,1,Negative
1340,cigarette smoke booked hotel reasonably priced...,2,Negative
2618,poor service poor management looking tired boo...,2,Negative


In [19]:
#randomize
df_all = df_all.sample(frac = 1)
df_all = df_all.reset_index(drop=True)

In [20]:
df_all.sample(5)

,Review,Rating,Sentiment
730,bumped hotel sister property hotel saint louis...,2,Negative
2098,"honeymoon heaven, husband just returned honeym...",5,Positive
4707,elegant hotel beautiful madrid westin palace l...,5,Positive
6324,"wow excellent value money- amazing wedding, 34...",5,Positive
1222,"loved hotel hotel conveniences home, including...",5,Positive


**Split test train**

In [21]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split(df_all.Review, df_all.Sentiment)

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
x_train_vec = v.fit_transform(X_train)
x_test_vec = v.transform(X_test)


**Model**

In [23]:
from sklearn import svm
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(x_train_vec , Y_train )

SVC(kernel='linear')

Accuracy

In [24]:
clf_svm.score(x_test_vec,Y_test)

0.9496268656716418

In [25]:
from sklearn.metrics import f1_score
f1_score(Y_test , clf_svm.predict(x_test_vec),average=None)

array([0.9487666 , 0.95045872])

In [27]:
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(random_state=99)
logreg.fit(x_train_vec,Y_train)

LogisticRegression(random_state=99)

In [28]:
logreg.score(x_test_vec,Y_test)

0.9552238805970149

In [29]:
f1_score(Y_test , logreg.predict(x_test_vec),average=None)

array([0.95437262, 0.95604396])

Pickling model file for deployment

In [30]:
import pickle

In [31]:
pickle.dump(logreg, open('hotel_reviews_1.pkl','wb'))

In [32]:
pickle.dump(clf_svm, open('hotel_reviews_2.pkl','wb'))

In [34]:
pickled_model=pickle.load(open('hotel_reviews_1.pkl','rb'))

In [39]:
pickled_model_1=pickle.load(open('hotel_reviews_2.pkl','rb'))

In [41]:
#Prediction
pickled_model.predict(x_test_vec)

array(['Negative', 'Positive', 'Positive', ..., 'Positive', 'Negative',
       'Negative'], dtype=object)

In [42]:
pickled_model_1.predict(x_test_vec)

array(['Negative', 'Positive', 'Positive', ..., 'Positive', 'Negative',
       'Negative'], dtype=object)